In [18]:
import tensorflow as tf
import os
import numpy as np
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
import math

In [2]:
DIR = os.getcwd() + "/train-val-test/db14/window-size-10/lag2/all-features/"

In [33]:
X_train = np.load(DIR + "pca-X_train.npy")
y_train = np.load(DIR + "y_train.npy")
X_val = np.load(DIR + "pca-X_val.npy")
y_val = np.load(DIR + "y_val.npy")
X_test = np.load(DIR + "pca-X_test.npy")
y_test = np.load(DIR + "y_test.npy")

In [27]:
X_train.shape

(5791, 1, 28, 28)

In [ ]:
with tf.device("/gpu:0"):
    X_test = tf.convert_to_tensor(X_test)
    y_test = tf.convert_to_tensor(y_test)

In [28]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=os.getcwd() + "/cnn-tf/saved_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [29]:
input_details

[{'name': 'serving_default_actual_input:0',
  'index': 0,
  'shape': array([ 1,  1, 28, 28], dtype=int32),
  'shape_signature': array([ 1,  1, 28, 28], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [38]:
def eval_cnn(data, labels):
    correct = 0
    total = 0
    preds = []

    # Test model on random input data.
    for i in range(data.shape[0]):
        input_shape = input_details[0]['shape']
        input_data = np.array(data[i,:,:,:][None,:,:,:], dtype=np.float32) #np.array(np.random.random_sample(input_shape), dtype=np.float32)
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()

        # The function `get_tensor()` returns a copy of the tensor data.
        # Use `tensor()` in order to get a pointer to the tensor.
        output_data = interpreter.get_tensor(output_details[0]['index'])[0]
        #print(output_data)
        #print(tf.math.argmax(abs(output_data), 1))
        #print(tf.math.argmax(abs(output_data), 1))

        pred = np.where(output_data == np.max(output_data))[0][0]
        #print(pred)
        preds.append(pred)
        acc = labels[i]
        #print(pred)
        #print(acc)
        #print()
        if abs(pred - acc) < 0.1:
            correct += 1
        total += 1

    print(correct/total)
    print(confusion_matrix(preds, labels))
    print()

In [39]:
eval_cnn(X_train, y_train)
eval_cnn(X_val, y_val)
eval_cnn(X_test, y_test)

0.9605066206102476
[[14830   590]
 [   96  1854]]

0.9345536176826109
[[4873  292]
 [  87  539]]

0.9292004835088931
[[4871  305]
 [ 105  510]]

